# Data Cleaning for IMDB Dataset

Author: Jennifer Le
Date: 1/22/25

In [2]:
import pandas as pd

'''
The code below was used to read in the original titles dataset.
To save space, this 11 million row, nearly 1GB dataset is replaced by a
smaller filtered dataset. 

# Read in original dataset and delete irrelevant columns and rows
df = pd.read_csv("title.basics.tsv", delimiter='\t', low_memory=False)
df = df.drop(columns=['isAdult','originalTitle'])

# Remove rows where titleType is in list, the ~ symbol negates the isin
print(df['titleType'].unique())
df = df[
    ~df["titleType"].isin(
        ['tvEpisode','video','videoGame','tvPilot','tvSpecial']
    )
]
df.to_csv("titles_reduced.csv",index=False)
'''

'\nThe code below was used to read in the original titles dataset.\nTo save space, this 11 million row, nearly 1GB dataset is replaced by a\nsmaller filtered dataset. \n\n# Read in original dataset and delete irrelevant columns and rows\ndf = pd.read_csv("title.basics.tsv", delimiter=\'\t\', low_memory=False)\ndf = df.drop(columns=[\'isAdult\',\'originalTitle\'])\n\n# Remove rows where titleType is in list, the ~ symbol negates the isin\nprint(df[\'titleType\'].unique())\ndf = df[\n    ~df["titleType"].isin(\n        [\'tvEpisode\',\'video\',\'videoGame\',\'tvPilot\',\'tvSpecial\']\n    )\n]\ndf.to_csv("titles_reduced.csv",index=False)\n'

In [2]:
# add files from google drive to correct folder to 
# run code block without file-not-found error

df_titles = pd.read_csv('../../titles_reduced.csv')
df_ratings = pd.read_csv('../../ratings.tsv', delimiter='\t', low_memory=False)
df_crew = pd.read_csv('../../crew.tsv', delimiter='\t', low_memory=False)

print(df_titles.shape)
print(df_ratings.shape)
print(df_crew.shape)

df_combined = pd.merge(df_titles, df_ratings, on='tconst', how='inner')
df_combined = pd.merge(df_combined, df_crew, on='tconst', how='inner')

# replace null values with 0 or empty space
df_combined['startYear'] = df_combined['startYear'].replace({'\\N': '0'}) 
df_combined['endYear'] = df_combined['endYear'].replace({'\\N': '0'})        
df_combined['runtimeMinutes'] = df_combined['runtimeMinutes'].replace({'\\N':'0'})
df_combined['genres'] = df_combined['genres'].replace({'\\N':''}) 
df_combined['directors'] = df_combined['directors'].replace({'\\N':''}) 
df_combined['writers'] = df_combined['writers'].replace({'\\N':''}) 

# convert types to save space
df_combined['endYear'] = df_combined['endYear'].astype('int32')   
df_combined['runtimeMinutes'] = df_combined['runtimeMinutes'].astype('int32')       
df_combined['startYear'] = df_combined['startYear'].astype('int32')  

df_combined.to_csv('result.csv', index=False)
print(df_combined.shape)
print(df_combined.dtypes)

df_combined.to_csv('imdb_dataset_original.csv', index=False)

(2238148, 7)
(1526047, 3)
(11401626, 3)


In [11]:
df_combined = pd.read_csv('imdb_dataset_original.csv')
df_names = pd.read_csv('../../names.csv', low_memory=False)

In [13]:
# split columns containing multiple values 
df_genres_split = df_combined['genres'].str.split(pat=',',expand=True, n=2)
df_directors_split = df_combined['directors'].str.split(pat=',',expand=True, n=3)
df_writers_split = df_combined['writers'].str.split(pat=',',expand=True, n=3)
df_writers_split.to_csv('result.csv',index=False)

# add new columns for the splited genres
df_combined['genres'] = df_genres_split[0]
df_combined.insert(7, 'genre2', df_genres_split[1])
df_combined.insert(8,'genre3',df_genres_split[2])

# add new columns for the splited directors
df_combined['directors'] = df_directors_split[0]
df_combined.insert(12, 'director2', df_directors_split[1])
df_combined.insert(13, 'director3', df_directors_split[2])

# add new columns for the splited writers
df_combined['writers'] = df_writers_split[0]
df_combined['writer2'] = df_writers_split[1]
df_combined['writer3'] = df_writers_split[2]

# change column names
df_combined = df_combined.rename(columns={
        'directors':'director1',
        'writers':'writer1',
        'genres':'genre1'
    }
)

# function to replace name id with actual name
def replace_id_with_names(df_combined, df_names, column_name):
    df_combined = pd.merge(df_combined,df_names, 
                        left_on=column_name, right_on='nconst',how ='left')

    df_combined[column_name] = df_combined['primaryName']
    df_combined = df_combined.drop(columns=['nconst','primaryName'])

    return df_combined

# replace id of directors and writers
df_combined = replace_id_with_names(df_combined, df_names, 'director1')
df_combined = replace_id_with_names(df_combined, df_names, 'director2')
df_combined = replace_id_with_names(df_combined, df_names, 'director3')
df_combined = replace_id_with_names(df_combined, df_names, 'writer1')
df_combined = replace_id_with_names(df_combined, df_names, 'writer2')
df_combined = replace_id_with_names(df_combined, df_names, 'writer3')

df_combined.to_csv('imdb_dataset_transformed.csv',index=False)